To run this yourself, make a copy or open in playground mode.

# Install NEURON, update matplotlib
The matplotlib update lets us specify `vertical_axis`

In [ ]:
!pip install neuron
!pip install --upgrade matplotlib

# Load the morphology definition

In [ ]:
!wget https://raw.githubusercontent.com/ramcdougal/dentategranulevideo/master/n275.hoc

--2022-03-20 01:50:21--  https://raw.githubusercontent.com/ramcdougal/dentategranulevideo/master/n275.hoc
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37639 (37K) [text/plain]
Saving to: ‘n275.hoc’

n275.hoc            100%[===================>]  36.76K  --.-KB/s    in 0.003s  

2022-03-20 01:50:21 (10.4 MB/s) - ‘n275.hoc’ saved [37639/37639]



# Setup the model

In [ ]:
from neuron import h
import matplotlib.pyplot as plt
import tqdm
from IPython.display import HTML
from base64 import b64encode
from neuron.units import ms, mV

plt.rcParams["figure.figsize"] = (6, 6)
h.load_file("stdrun.hoc")
h.load_file("n275.hoc")

1.0

In [ ]:
STOP_TIME = 45 * ms
SAVE_EVERY = 0.25 * ms

In [ ]:
for sec in h.allsec():
  sec.nseg = 21
  if 'dend' not in sec.name():
    sec.insert(h.hh)

In [ ]:
# setup current pulses to trigger APs
fire_times = [0 * ms, 15 * ms, 23 * ms, 31 * ms]
iclamps = []
for time in fire_times:
    iclamp = h.IClamp(h.soma[0](0.5))
    iclamp.delay = time
    iclamp.amp = 2
    iclamp.dur = 0.5 * ms
    iclamps.append(iclamp)

# Simulation control and image saving

In [ ]:
def neuron_images():
  # rotation
  ps = h.PlotShape(False)
  ps.plot(plt)
  for theta in range(0, 360, 9):
    plt.gca().view_init(0, theta, vertical_axis="y")
    yield 
  plt.close()
  ps.variable("v")
  ps.scale(-80, 50)
  # now let's run the sim, plt on a new figure every SAVE_EVERY, then yield
  h.finitialize(-65 * mV)
  for i in range(200):
    h.continuerun(i * SAVE_EVERY)
    ps.plot(plt)
    if i < 40:
       theta = 9 * i
    else:
      theta = 0
    plt.gca().view_init(0, theta, vertical_axis="y")
    yield
    plt.close()



In [ ]:
def save_all_images():
  for i, _ in tqdm.tqdm(enumerate(neuron_images())):
    plt.savefig(f"{i:04d}.png")

This counts to 40 + 200 = 240. Runs in about twelve minutes. The first 40 is relatively fast because it's only rotating an existing plot, not simulating or making a new plot.

In [ ]:
save_all_images()

240it [10:26,  2.61s/it]


# Put it all together into an MP4

In [ ]:
!ffmpeg -r 20 -i %04d.png -vcodec libx264 -crf 25 -pix_fmt yuv420p neuron_movie.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

# Let's look at the MP4

In [ ]:
# adapted from https://stackoverflow.com/questions/57377185/how-play-mp4-video-in-google-colab
def show_video(video_path):
  with open(video_path, "r+b") as f:
    video_url = f"data:video/mp4;base64,{b64encode(f.read()).decode()}"
  return HTML(f"<video width=640 controls><source src='{video_url}'></video>")

In [ ]:
show_video("neuron_movie.mp4")